# Biquad transfer function (Second order stages)

A second order systen (sos), a recursive linear filter is often represented as

$$ H(z) = \frac{N(z)}{D(z)} = \frac{b_{0} + b_{1}z^{-1} + b_{2}z^{-2}}{a_{0} + a_{1}z^{-1} + a_{2}z^{-2}}, $$

With $N(z)$ and $D(z)$ the numerator and denominator respectively. For example on the Wikipedia page [Digital biquad filter](https://en.wikipedia.org/wiki/Digital_biquad_filter). A digital filter may consist of multiple of these second order stages.

However, sometimes (e.g. [1] both $N(z)$ and $D(z)$ are multiplied by $z^2$

$$ H(z) = \frac{a_{0} + a_{1}z + a_{2}z^2}{b_{0} + b_{1}z + b_{2}z^2}, $$


We know that $b_2$ is always 1 and we can make $a_2$ 1 by taking $H = a_2$ and multiplying the numerator with $\frac{1}{a_2}$. 

$$ H(z) = H \frac{a_{0} + a_{1}z + z^2}{b_{0} + b_{1}z + z^2}, $$

Cascading $J$ of these second-order stages will result in the total transfer function

$$ H(\mathbf{c}, z)  = H_0 \prod^J_{m=1}\frac{a_{0m} + a_{1m}z + z^2}{b_{0m} + b_{1m}z + z^2}, $$


with filter coefficients $$ \mathbf{c} = [a_{01} \; a_{11} \; b_{01} \; b_{11} \; \dots  \; b_{0J} \; b_{1J} \; H_0]^T. $$

And with $H_0 = \prod^J_{m=1} H_m$



[1]: Andreas Antoniou - Digital Filters: Analysis, Design, and Signal Processing Applications


Let's use scipy.signal to generate a lowpass iir filter with a single second order stage:

In [1]:
from scipy import signal
filt = signal.iirfilter(2, 100, btype='low',
                       analog=False, rs=60, rp=0.01,
                        ftype='ellip', fs=1000,output='sos')

In [2]:
print(filt)

[[0.31018857 0.61926624 0.31018857 1.         0.05679353 0.18427788]]


But which coefficients are which? We can wrap it in a named tuple

In [3]:
from collections import namedtuple
StandardSOS = namedtuple('StandardSOS',
        ['b0', 'b1', 'b2','a0', 'a1', 'a2'])

In [4]:
sos = StandardSOS(*filt[0])
sos

StandardSOS(b0=0.31018857356059193, b1=0.6192662419329938, b2=0.3101885735605919, a0=1.0, a1=0.05679352648032663, a2=0.18427787664122666)

In [15]:
import numpy as np

def dsp2antoniou(c):
    """ Translates second order filter sections from DSP-convention
    format (StandardSOS) to optimized Antoniou format.
    
    parameters
    ----------
    
    c: arraylike
        Second order stage with the following order:
        
        [b0, b1, b2, a0, a1, a2] such that
        
           b0 + b1*z^-1 + b2*z^-2
    H(z) = ----------------------
           1  + a1*z^-1 + a2*z^-2
            
    returns
    -------
    
    (coeffs, H): tuple
        coefs = [a0, a1, b1, b2] and H the multiplier such that
        
              a0 + a1z + z^2
    H(z) = H  --------------
              b0 + b1z + z^2
        
    """
    
    c = StandardSOS(*c)

    H = c.b0
    # numerator
    a0 = c.b2 / H
    a1 = c.b1 / H
    # a2 = c.b0 / c.b0 = 1
    
    # denominator
    b0 = c.a2
    b1 = c.a1
    # b2 = c.a0 = 1
    
    return [a0, a1, b0, b1], H


def simplify_biquad_filters(system):
    """
    Converts array of N standard second order stages (SOS) such as generated by scipy.signal into
    optimized filter coefficient format.
    
    parameters
    ----------
    
    system: list of standard format sos
            
            [[b0, b1, b2, a0, a1, a2], [b0, b1, b2, a0, a1, a2], ... ]
    
    returns
    -------
    
    coeffs: ndarray
        [a_01 a_11 b_01 b_11 ... b_0N b_1N H0]
        
    Where a_01 a_11 b_01 b_11 are the coefficients for stage 1 and a_02 a_12 b_02 b_12 for stage 2, etc. 

    
                  N   a0j + a1jz + z^2
    H(c, z) = H0  ∏   ----------------, 
                 j=0  b0j + b1jz + z^2
                 
    for j = range(1, N)
        
    """
    a = [dsp2antoniou(sos) for sos in system]
    coeffs, Hm = list(zip(*a))
    print(Hm)
    H0 = np.product(Hm)
    return np.r_[np.concatenate(coeffs), H0]

In [93]:
dsp2antoniou(sos)

([1.0000000000000002,
  0.8730113243817196,
  0.4807911839677346,
  -1.3599112704531684],
 0.0025610472568130785)

In [9]:
filt = signal.iirfilter(5, 1, btype='low',
                       analog=False, rs=60, rp=0.01,
                        ftype='ellip', fs=1000,output='sos')
filt

array([[ 3.75305631e-05,  3.75305631e-05,  0.00000000e+00,
         1.00000000e+00, -9.94497315e-01,  0.00000000e+00],
       [ 1.00000000e+00, -1.99931626e+00,  1.00000000e+00,
         1.00000000e+00, -1.99181886e+00,  9.91861638e-01],
       [ 1.00000000e+00, -1.99972615e+00,  1.00000000e+00,
         1.00000000e+00, -1.99721455e+00,  9.97274263e-01]])

In [14]:
simplify_biquad_filters(filt)

(3.753056310358706e-05, 1.0, 1.0)


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 12 and the array at index 1 has size 1

In [106]:
np.poly?

Signature: np.poly(seq_of_zeros)
Docstring:
Find the coefficients of a polynomial with the given sequence of roots.

.. note::
   This forms part of the old polynomial API. Since version 1.4, the
   new polynomial API defined in `numpy.polynomial` is preferred.
   A summary of the differences can be found in the
   :doc:`transition guide </reference/routines.polynomials>`.

Returns the coefficients of the polynomial whose leading coefficient
is one for the given sequence of zeros (multiple roots must be included
in the sequence as many times as their multiplicity; see Examples).
A square matrix (or array, which will be treated as a matrix) can also
be given, in which case the coefficients of the characteristic polynomial
of the matrix are returned.

Parameters
----------
seq_of_zeros : array_like, shape (N,) or (N, N)
    A sequence of polynomial roots, or a square array or matrix object.

Returns
-------
c : ndarray
    1D array of polynomial coefficients from highest to lowest degree

In [108]:
np.poly(np.array([1]))

array([ 1., -1.])